In [1]:
import argparse
from datetime import datetime
import json
import logging
import os
import math
import sys
import time
import topologyqueries,linkedlist,update_topology,spanning_tree

from gridappsd import GridAPPSD, DifferenceBuilder, utils, topics as t
from gridappsd.topics import simulation_input_topic, simulation_log_topic, simulation_output_topic


#model_mrid = sim_request["power_system_config"]["Line_name"]
#model_mrid = "_C125761E-9C21-4CA9-9271-B168150DE276" #ieee13training
model_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #final9500node
#model_mrid = "_AAE94E4A-2465-6F5E-37B1-3E72183A4E44" #test9500
#model_mrid = "_5B816B93-7A5F-B64C-8460-47C17D6E4B0F" #ieee13assets
#model_mrid="_C1C3E687-6FFD-C753-582B-632A27E28507" # IEEE 123

requestedTime=1570041125



global BaseConnDict,BaseTermDict,TermList, NodeList
global XfmrKeys,XfmrDict,SwitchKeys,SwitchDict,DG_query

os.environ['GRIDAPPSD_USER'] = 'app_user'
os.environ['GRIDAPPSD_PASSWORD'] = '1234App'

gapps = GridAPPSD()
assert gapps.connected

In [2]:
# Query for ACLineSegment, Transformerend, Switches, Breakers, Reclosers, Fuses, Sectionalisers,SynchronousMachine,TopologicalNode
StartTime = time.process_time()

topologyqueries.getallqueries(gapps,model_mrid)
Line_query=topologyqueries.Line_query
XfmrDict=topologyqueries.XfmrDict
XfmrKeys=topologyqueries.XfmrKeys
SwitchDict=topologyqueries.SwitchDict
SwitchKeys=topologyqueries.SwitchKeys
DG_query=topologyqueries.DG_query
Cap_query=topologyqueries.Cap_query
NodeQuery=topologyqueries.Node_query


print( time.process_time() - StartTime, "seconds")


0.23937103600000004 seconds


In [3]:
MeasurementQuery = topologyqueries.measurement_query(gapps, model_mrid)

ConnNodeEquipDict = {}

StartTime = time.process_time()

for i0 in range(len( NodeQuery)):
    node= NodeQuery[i0]['cnid']['value']

    ConnNodeEquipDict[node] = {}
    ConnNodeEquipDict[node]['name'] =  NodeQuery[i0]['busname']['value']
    ConnNodeEquipDict[node]['TopologicalNode'] =  NodeQuery[i0]['tpnid']['value']
    
    ConnNodeEquipDict[node]['ACLineSegment'] = []
    ConnNodeEquipDict[node]['Breaker'] = []
    ConnNodeEquipDict[node]['EnergyConsumer'] = []
    ConnNodeEquipDict[node]['Fuse'] = []
    ConnNodeEquipDict[node]['LinearShuntCompensator'] = []
    ConnNodeEquipDict[node]['LoadBreakSwitch'] = []
    ConnNodeEquipDict[node]['PowerTransformer'] = []
    ConnNodeEquipDict[node]['Recloser'] = []
    ConnNodeEquipDict[node]['TransformerEnd'] = []    
    ConnNodeEquipDict[node]['SynchronousMachine'] = []
    ConnNodeEquipDict[node]['PowerElectronicsConnection'] = []
    ConnNodeEquipDict[node]['measurements'] = []

print('Processed ', i0+1, 'nodes in ', time.process_time() - StartTime, "seconds")

NameError: name 'Node_query' is not defined

In [ ]:
StartTime = time.process_time()

for i1 in range(len(MeasurementQuery)):
    node = MeasurementQuery[i1]['cnid']['value']
    ConnNodeEquipDict[node]['measurements'].append(MeasurementQuery[i1]['measid']['value'])
    if MeasurementQuery[i1]['eqid']['value'] not in ConnNodeEquipDict[node][MeasurementQuery[i1]['eqtype']['value']]:
        ConnNodeEquipDict[node][MeasurementQuery[i1]['eqtype']['value']].append(MeasurementQuery[i1]['eqid']['value'])
        
print('Processed ', i1+1, 'measurement objects in ', time.process_time() - StartTime, "seconds")

In [ ]:
# Build LinkNet Lists of entire feeder
[ConnNodeDict,TerminalsDict,TermList,NodeList]=linkedlist.build_linked_list(Line_query, XfmrDict, XfmrKeys, DG_query, Node_query)

# Build second LinkNet List for only single voltage levels
[MVConnNodeDict,MVTerminalsDict,MVTermList,MVNodeList]=linkedlist.build_linked_list(Line_query, {}, [], DG_query, Node_query)

In [ ]:
for i1 in range(len(SwitchKeys)):

    ConnNodeDict[SwitchDict[SwitchKeys[i1]]['node1']]['switch.mRID'] = SwitchKeys[i1]
    ConnNodeDict[SwitchDict[SwitchKeys[i1]]['node2']]['switch.mRID'] = SwitchKeys[i1]
    MVConnNodeDict[SwitchDict[SwitchKeys[i1]]['node1']]['switch.mRID'] = SwitchKeys[i1]
    MVConnNodeDict[SwitchDict[SwitchKeys[i1]]['node2']]['switch.mRID'] = SwitchKeys[i1]
    
    
    

In [ ]:
BaseConnDict = json.dumps(ConnNodeDict)
BaseTermDict = json.dumps(TerminalsDict)

In [ ]:
[TerminalsDict, ConnNodeDict]=update_topology.topology_update(BaseConnDict,BaseTermDict,SwitchDict,SwitchKeys,TermList)

SubstationRoots = []
# Iterate through all substation transformers
for i6 in range(len(XfmrKeys)):

    # Identify if distribution substation transformer
    if ((int(XfmrDict[XfmrKeys[i6]]['volt1']) >= 34000 and 34000 >= int(XfmrDict[XfmrKeys[i6]]['volt2']) > 1000) or 
        (int(XfmrDict[XfmrKeys[i6]]['volt2']) >= 34000 and 34000 >= int(XfmrDict[XfmrKeys[i6]]['volt1']) > 1000)):
        SubstationRoots.append(XfmrKeys[i6])
        
[SubTree, ConnNodeDict] = SpanningTree(ConnNodeDict, TerminalsDict, NodeList, XfmrDict, SubstationRoots, 'single')

In [ ]:
ConnNodeDict = json.loads(BaseConnDict)
TerminalsDict = json.loads(BaseTermDict)

In [ ]:
[AllTrees, ConnNodeDict] = spanning_tree(ConnNodeDict, TerminalsDict, NodeList, SwitchDict, SwitchKeys, 'all')


In [ ]:
DistAppStruct = {}
DistAppStruct['feeders'] = {}
DistAppStruct['feeders'][model_mrid] = {}
DistAppStruct['feeders'][model_mrid]['addressabe_equipment'] = []
DistAppStruct['feeders'][model_mrid]['unaddressabe_equipment'] = []
DistAppStruct['feeders'][model_mrid]['switch_areas'] = []


In [ ]:
[MVTrees, MVConnNodeDict] = spanning_tree(MVConnNodeDict, TerminalsDict, MVNodeList, SwitchDict, SwitchKeys, 'all')


In [ ]:
[MVTrees[SwitchKeys[i1]][0]]

In [ ]:
ConnNodeDict[MVTrees[SwitchKeys[i1]][0]]

In [ ]:
ConnNodeDict['_5B9656C8-717E-4707-A2CA-A8994EC647E4']

In [ ]:
for i1 in range(len(SwitchKeys)):
    SwitchArea = {}
    SwitchArea['boundary_switches'] = []
    SwitchArea['addressable_equipment'] = []
    SwitchArea['unaddressable_equipment'] = []
    SwitchArea['connectivity_node'] = []
    SwitchArea['secondary_areas'] = []
    
    for i2 in range(len(MVTrees[SwitchKeys[i1]])):
        
        SwitchArea['boundary_switches'].append(ConnNodeDict[MVTrees[SwitchKeys[i1]][i2]]['switch.mrid'])
        

In [ ]:
switch = '_1962A006-8FA7-403E-A06F-DCBFE991E771'
for i8 in range(len(Tree[switch])):
    print(ConnNodeDict[Tree[switch][i8]]['name'])

In [ ]:
SwitchArea = {}
SwitchArea['boundary_switches'] = []
SwitchArea['boundary_switches'].append([])

In [ ]:
def spanning_tree(ConnNodeDict, TerminalsDict, NodeList, EquipDict, RootKeys, Scope):
    Tree={}
    #ProcessedNodes=[]
    TotalNodes=0
    StartTime = time.process_time()

    # Iterate through all substation transformers
    for i6 in range(len(RootKeys)):

        FirstNode = 1 
        LastNode = 2
        # Set switch as rootnode 
        if 'node' in EquipDict[RootKeys[i6]]: # Check if 1-terminal object
            Tree[RootKeys[i6]]=[EquipDict[RootKeys[i6]]['node']]
        else: # Then 2-terminal object
            Tree[RootKeys[i6]]=[EquipDict[RootKeys[i6]]['node1'], EquipDict[RootKeys[i6]]['node2']]

        while LastNode != FirstNode:
            NextTerm = ConnNodeDict[Tree[RootKeys[i6]][FirstNode]]['list']
            FirstNode = FirstNode + 1
            while NextTerm != 0:
                # Get next node and terminal for current node
                NextNode = TerminalsDict[TermList[NextTerm-1]]['far']
                NextTerm = TerminalsDict[TermList[NextTerm-1]]['next']
                node = NodeList[NextNode-1]
                not_in_tree = True

                if Scope == 'all': 
                    for i7 in range(len(Tree.keys())):
                        if node in Tree[RootKeys[i7]]:
                            not_in_tree = False
                            break
                else: 
                    if NodeList[NextNode-1] in Tree[RootKeys[i6]]: 
                        not_in_tree = False# Check if node already in all other tree
                    

                # Add node if not in another tree        
                if not_in_tree:       
                    if ConnNodeDict[node]['nomv']:
                        if int(ConnNodeDict[node]['nomv']) < 34000 and int(ConnNodeDict[node]['nomv']) > 1000:
                            Tree[RootKeys[i6]].append(NodeList[NextNode-1])
                            LastNode = LastNode + 1                       
                    else:
                        Tree[RootKeys[i6]].append(NodeList[NextNode-1])
                        LastNode = LastNode + 1

        NodesInTree=len(Tree[RootKeys[i6]])
        
        
        print("Processed topology from  ", RootKeys[i6], ' with ', NodesInTree, " buses")

        #print("Processed topology from  ", EquipDict[RootKeys[i6]]['name'], " with id ", RootKeys[i6], ' with ', NodesInTree, " buses")
        TotalNodes=TotalNodes+NodesInTree

    print("Processed ", len(Tree.keys()), "topology trees containing ", TotalNodes, " buses in ", time.process_time() - StartTime, "seconds")

    return Tree, ConnNodeDict


In [7]:
QueryLines="""
    PREFIX r:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX c:  <http://iec.ch/TC57/CIM100#>
    SELECT ?name ?bus1 ?bus2 ?id ?tname1 ?term1 ?tname2 ?term2 ?node1 ?node2 ?tpnode1 ?tpnode2 (group_concat(distinct ?phs;separator="") as ?phases) WHERE {
    SELECT ?name ?bus1 ?bus2 ?phs ?id ?tname1 ?term1 ?tname2 ?term2 ?node1 ?node2 ?tpnode1 ?tpnode2 WHERE {
    VALUES ?fdrid {"%s"}  
    VALUES ?id {'_869F4EB9-5E95-4B59-B76B-B04FAE942C85'}
    ?fdr c:IdentifiedObject.mRID ?fdrid.
    ?s r:type c:ACLineSegment.
    ?s c:Equipment.EquipmentContainer ?fdr.
    ?s c:IdentifiedObject.name ?name.
    ?s c:IdentifiedObject.mRID ?id.
    ?t1 c:Terminal.ConductingEquipment ?s.
    ?t1 c:ACDCTerminal.sequenceNumber "1".
    ?t1 c:Terminal.ConnectivityNode ?cn1. 
    ?t1 c:IdentifiedObject.name ?tname1.
    ?cn1 c:IdentifiedObject.name ?bus1.
    ?cn1 c:ConnectivityNode.TopologicalNode ?tp1.
    ?t2 c:Terminal.ConductingEquipment ?s.
    ?t2 c:ACDCTerminal.sequenceNumber "2".
    ?t2 c:Terminal.ConnectivityNode ?cn2. 
    ?t2 c:IdentifiedObject.name ?tname2.
    ?cn2 c:ConnectivityNode.TopologicalNode ?tp2.
    ?cn2 c:IdentifiedObject.name ?bus2.
    bind(strafter(str(?t),"#") as ?tid).
        bind(strafter(str(?t1), "#") as ?term1) 
        bind(strafter(str(?t2), "#") as ?term2)
        bind(strafter(str(?cn1), "#") as ?node1)
        bind(strafter(str(?cn2), "#") as ?node2)
        bind(strafter(str(?tp1), "#") as ?tpnode1)
        bind(strafter(str(?tp2), "#") as ?tpnode2)
            OPTIONAL {?acp c:ACLineSegmentPhase.ACLineSegment ?s.
            ?acp c:ACLineSegmentPhase.phase ?phsraw.
            bind(strafter(str(?phsraw),"SinglePhaseKind.") as ?phs) }

    } ORDER BY ?name ?phs
    }
    GROUP BY ?name ?bus1 ?bus2 ?id ?tname1 ?term1 ?tname2 ?term2 ?node1 ?node2 ?tpnode1 ?tpnode2
    ORDER BY ?name
    """%model_mrid
results = gapps.query_data(query = QueryLines, timeout = 60)
Line_query = results['data']['results']['bindings']

In [8]:
Line_query

[]